# Interactive seq2seq
Applying our trained encoder-decoder model so that it can produce and respond to dialogue in context. We will need to load the model, seq2seq_baseline.ckpt, which was trained on the NLP cluster with 2500 iterations and batch size of 1028.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
import random
import json
from tensorflow.python.layers.core import Dense

In [2]:
import sys
sys.path.append("../src/models/")
sys.path.append('../src/models/agents/')
sys.path.append('../src/data/')
from agent import Agent
from parse import FBParser

### Set global variables


In [3]:
train_iterations = 3
learning_rate = 0.1
max_input_length = 6 # length of goals list
max_output_length = 20
unk_threshold = 20

### Initialize parser and agent
Using the Agent() class which subclasses the basic TfEncoderDecoder for dialogue.

In [4]:
parser = FBParser(unk_threshold=unk_threshold,
                  input_directory="../data/raw/",
                  output_directory="../data/processed/")
print("Vocab size: {}".format(parser.vocab_size))

Vocab size: 502


In [5]:
agent = Agent(vocab=parser.vocab,
              max_iter=train_iterations,
              eta=learning_rate,
              max_input_length=max_input_length,
              max_output_length=max_output_length,
              hidden_dim=64)

### Model training

In [6]:
train_data = []
with open("../data/processed/train.txt", "r") as train_file:
    for line in train_file:
        train_example = json.loads(line)
        train_data.append((
            train_example["input"],
            train_example["output"][0].split()))

In [7]:
X, y = zip(*train_data)

In [8]:
agent.fit(X, y, save_path="../models/seq2seq3")

Iteration 3: loss: 46.062458515167236

Model saved in path: ../models/seq2seq3


In [75]:
with agent.sess.as_default():
    print(agent.sess.graph.get_collection("trainable_variables")[0].eval())

[[-0.06525127  0.01688811  0.05999434 ... -0.08155969 -0.02386533
  -0.08287811]
 [ 0.02278102  0.04003168 -0.03195427 ...  0.05372681 -0.02829458
   0.04099403]
 [-0.00512103  0.0475482   0.03017184 ... -0.08863152 -0.08470751
  -0.05197031]
 ...
 [ 0.01693503  0.02145671  0.0007079  ... -0.09613955  0.02751628
  -0.09628318]
 [ 0.10384274 -0.08813189  0.00460204 ... -0.04744328 -0.03279633
   0.02506361]
 [ 0.08731178  0.05644462 -0.04629318 ...  0.03718802 -0.06587479
   0.06116211]]


Load the saved model:

In [10]:
# tf.reset_default_graph()
# newAgent = Agent(vocab=parser.vocab,
#               max_iter=train_iterations,
#               eta=learning_rate,
#               max_input_length=max_input_length,
#               max_output_length=max_output_length,
#               hidden_dim=64)
# newAgent.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
# newAgent.build_graph()
# newAgent.sess.run(tf.global_variables_initializer())

In [47]:
# saver = tf.train.import_meta_graph('../models/seq2seq.ckpt.meta', clear_devices=True)

In [9]:
# saver.restore(agent.sess, '../models/seq2seq.ckpt')

Print elements of the graph:

In [9]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['encoder_inputs',
 'encoder_lengths',
 'decoder_inputs',
 'decoder_targets',
 'decoder_lengths',
 'EmbedSequence/embeddings/Initializer/random_uniform/shape',
 'EmbedSequence/embeddings/Initializer/random_uniform/min',
 'EmbedSequence/embeddings/Initializer/random_uniform/max',
 'EmbedSequence/embeddings/Initializer/random_uniform/RandomUniform',
 'EmbedSequence/embeddings/Initializer/random_uniform/sub',
 'EmbedSequence/embeddings/Initializer/random_uniform/mul',
 'EmbedSequence/embeddings/Initializer/random_uniform',
 'EmbedSequence/embeddings',
 'EmbedSequence/embeddings/Assign',
 'EmbedSequence/embeddings/read',
 'EmbedSequence/embedding_lookup/axis',
 'EmbedSequence/embedding_lookup',
 'random_uniform/shape',
 'random_uniform/min',
 'random_uniform/max',
 'random_uniform/RandomUniform',
 'random_uniform/sub',
 'random_uniform/mul',
 'random_uniform',
 'Variable',
 'Variable/Assign',
 'Variable/read',
 'embedding_lookup/axis',
 'embedding_lookup',
 'encoding_layer/Rank',
 'encodin

### Test data prediction
Make sure the saved model works correctly:

In [5]:
test_data = []
with open("../data/processed/test.txt", "r") as test_file:
    for line in test_file:
        test_example = json.loads(line)
        test_data.append((
            test_example["input"],
            test_example["output"][0].split()))

Get first twenty examples:

In [6]:
X_test, y_test = zip(*test_data[:20])

In [11]:
logits = agent.predict(X_test)
print('\nPredictions:\n')
for string in agent.output(logits, padding=" "):
    print(string)


Predictions:

<eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
<eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
<eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
<eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
<eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
<eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
<eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
<eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
<eos> <eos> <eos> <eos> <

In [16]:
# X = X_test[0]

Try concatenating string with inputs (issue with different embedding spaces):

In [17]:
# string = "THEM: i can have the hat and the ball"
# tokenized_string = string.split()
# indexed_string = []
# for word in tokenized_string:
#     indexed_string.append(agent.vocab.index(word))
# X = [X + indexed_string]
# print(X)

[[2, 2, 3, 2, 1, 0, 10, 11, 17, 18, 19, 52, 15, 19, 41]]


In [80]:
# X = np.asarray(list(X))
# x_lengths = [len(seq) for seq in X] # len(seq) == 6
# num_examples = len(X)
# length = len(X[0])

# # Resize X and x_lengths to match the size of inference_logits:
# X.resize((agent.batch_size, length))
# x_lengths = np.asarray(x_lengths)
# x_lengths.resize(agent.batch_size)

# answer_logits = agent.sess.run(agent.inference_logits, {
#     agent.encoder_inputs: X, 
#     agent.decoder_inputs: [indexed_string],
#     agent.encoder_lengths: x_lengths})[:1]

# for string in agent.output(answer_logits, padding=" "):
#     print(string)

THEM: i can have the hat and the hat , you can have the rest <eos> YOU: ok <eos> THEM:


### Loading saved agent (fails)
When we load the model, the predictions are significantly different and look unreasonable:

In [7]:
# From the NLP cluster: ../models/seq2seq.ckpt.meta
# Created locally: ../models/seq2seq.meta

tf.reset_default_graph()
new_agent = Agent(vocab=parser.vocab,
              max_iter=train_iterations,
              eta=learning_rate,
              max_input_length=max_input_length,
              max_output_length=max_output_length,
              hidden_dim=64)
new_agent.sess = tf.Session()
new_agent.build_graph()
new_agent.sess.run(tf.global_variables_initializer())
# saver = tf.train.import_meta_graph('../models/seq2seq.meta')
# saver.restore(new_agent.sess, '../models/seq2seq')

In [77]:
with new_agent.sess.as_default():
#     saver = tf.train.import_meta_graph('../models/seq2seq.meta')
#     saver.restore(new_agent.sess, '../models/seq2seq')
    print(new_agent.sess.graph.get_collection("trainable_variables")[0].eval())

[[-0.06270827  0.01239639  0.08007193 ... -0.06640103  0.09292398
   0.09548999]
 [-0.08993747 -0.00046965  0.07255783 ...  0.08008491 -0.05252932
  -0.04877758]
 [-0.02892955  0.04587383  0.00438385 ... -0.10265501  0.04483695
   0.032836  ]
 ...
 [-0.03772841  0.03908908  0.03669993 ... -0.02401637 -0.04016433
  -0.01054999]
 [ 0.02652953  0.03188018  0.02637091 ...  0.09641343  0.00352957
  -0.05047622]
 [ 0.01262499  0.00133081 -0.06215981 ...  0.05505873 -0.09190471
  -0.10097237]]


In [36]:
with new_agent.sess.as_default():
    print(new_agent.decoder_embedding_space.eval())

[[0.5630189  0.9546292  0.64599466 ... 0.61294866 0.7089076  0.77672875]
 [0.4098121  0.6093272  0.80525625 ... 0.41042948 0.7360611  0.8975309 ]
 [0.14562404 0.8105842  0.7939904  ... 0.05978286 0.5384811  0.24008536]
 ...
 [0.95700717 0.8537878  0.22956884 ... 0.27878475 0.07299829 0.14081085]
 [0.31523085 0.9224106  0.08257258 ... 0.36000538 0.6840925  0.23142505]
 [0.56795716 0.56745887 0.24331915 ... 0.5563756  0.912182   0.8349335 ]]


In [87]:
saver = tf.train.import_meta_graph('../models/seq2seq3.meta')
saver.restore(new_agent.sess, '../models/seq2seq3')
with new_agent.sess.as_default():
    print(new_agent.sess.graph.get_collection("trainable_variables")[0].eval())


INFO:tensorflow:Restoring parameters from ../models/seq2seq3
[[-0.02462596  0.05337482 -0.03597003 ... -0.04776893 -0.07873607
  -0.08314859]
 [-0.06342065  0.07026108  0.05910229 ... -0.04484149 -0.09270024
  -0.03258888]
 [-0.05534776  0.03549982 -0.09053169 ...  0.00167339 -0.00805367
  -0.0694021 ]
 ...
 [ 0.03875104 -0.06533718 -0.00296863 ...  0.07525247  0.03954347
  -0.05523655]
 [ 0.06748437 -0.03541719 -0.0211759  ...  0.0899797   0.06188979
   0.02779076]
 [-0.04769764  0.0371431  -0.02256215 ...  0.10079367 -0.06803156
  -0.02933522]]


In [92]:
with new_agent.sess.as_default():
    saver = tf.train.import_meta_graph('../models/seq2seq3.meta')
    saver.restore(new_agent.sess, '../models/seq2seq3')
    print(new_agent.sess.graph.get_collection("trainable_variables")[0].eval())

INFO:tensorflow:Restoring parameters from ../models/seq2seq3
[[ 0.00102177 -0.00259513 -0.02878302 ...  0.08825765  0.05260494
   0.08485631]
 [ 0.05216862  0.09489172  0.0333797  ...  0.00865414  0.05894401
   0.0136352 ]
 [-0.05372884 -0.09704466 -0.01118287 ...  0.07201279 -0.08176759
   0.06332421]
 ...
 [ 0.096106    0.03701653  0.0274476  ... -0.07601067  0.02503408
  -0.09112439]
 [ 0.03824776 -0.06067414  0.03799119 ...  0.00943223 -0.04537005
   0.07739425]
 [ 0.01495518 -0.05156463 -0.0430168  ...  0.04995845  0.00262113
   0.09828202]]


In [8]:
new_agent.load()

INFO:tensorflow:Restoring parameters from ../models/seq2seq3


In [9]:
logits = new_agent.predict(X_test)
print('\nPredictions:\n')
for string in new_agent.output(logits, padding=" "):
    print(string)


Predictions:

hat hat head item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 take item1=4 item1=4 item1=4 item1=4 item1=4
case head head head bad only only only only only only please means please please please please please please please
case case only only only only means means only only please please please please please please please please please please
hat head head head item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4
one head head head head head bad only only only only only only please please please please please please please
one head head head head head head never item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4 item1=4
case only only only only means means only only only please please please please please please please please please please
only only means only only only means only only please please please please please pl